### Implementação Básica de Filtragem Colaborativa baseada em Usuários

Filtragem colaborativa é um processo oriundo de sistemas de recomendação, onde o objetivo é filtrar uma lista de itens para sugerir a um usuário o item mais "relevante" dessa lista.

Itens são avaliados como "relevante" a partir de uma análise que busca por similaridades em um histórico de avaliações.

#### 1. Importação de Bibliotecas

In [1]:
import numpy as np

#### 2. Histórico de Avaliações

Usaremos dados de um histórico de avaliações para os processos subsequentes.

Para cada usuário, um array contém as avaliações daquele usuário para cada item de um catálogo:

`usuario = [item0, item1, item2, item3, ...]`

Caso um usuário não tenha avaliado um item, deixamos a avaliação como `0`.

In [2]:
NUM_USUARIOS = 4
NUM_ITENS = 5

usuario0 = [5, 1, 0, 2, 2]
usuario1 = [1, 5, 2, 5, 5]
usuario2 = [2, 0, 3, 5, 4]
usuario3 = [4, 3, 5, 3, 0]


Vamos transformar esses dados em uma matrix (um array 2D).

In [3]:
avaliacoes = [usuario0, usuario1, usuario2, usuario3]

matrix_avaliacoes = np.array(avaliacoes)

In [4]:
print(matrix_avaliacoes)

[[5 1 0 2 2]
 [1 5 2 5 5]
 [2 0 3 5 4]
 [4 3 5 3 0]]


Cada linha dessa matrix contém um vetor de todas avaliações de um usuário.

Podemos comparar esse vetores para ver o quão similar dois usuários são.

#### 3. Calculando a similaridade entre dois usuários.

Vamos definir uma função que pega dois usuários e retorna a similaridade entre eles.

In [5]:
def calcular_similaridade(vetor1: np.ndarray, vetor2: np.ndarray) -> float:
    """
    Calcula a similaridade entre dois vetores a partir do coseno do ângulo entre
    eles.

    Args:
        item1: numpy.ndarray do primeiro vetor
        item2: numpy.ndarray do segundo vetor

    Returns:
        similaridade: float correspondendo a similaridade entre os dois vetores
    """

    dot = np.dot(vetor1, vetor2)
    
    magnitude1 = np.linalg.norm(vetor1)
    magnitude2 = np.linalg.norm(vetor2)

    similaridade = dot / (magnitude1 * magnitude2)

    return similaridade

Com esses materiais, já conseguimos calcular a similaridade entre usuarios.

In [6]:
for i, usuario in enumerate(matrix_avaliacoes):
    for j, outro_usuario in enumerate(matrix_avaliacoes):
        sim = calcular_similaridade(usuario, outro_usuario)
        print(f"Similaridade entre Usuários {i} e {j}: {sim:.3g}")

Similaridade entre Usuários 0 e 0: 1
Similaridade entre Usuários 0 e 1: 0.575
Similaridade entre Usuários 0 e 2: 0.653
Similaridade entre Usuários 0 e 3: 0.647
Similaridade entre Usuários 1 e 0: 0.575
Similaridade entre Usuários 1 e 1: 1
Similaridade entre Usuários 1 e 2: 0.806
Similaridade entre Usuários 1 e 3: 0.64
Similaridade entre Usuários 2 e 0: 0.653
Similaridade entre Usuários 2 e 1: 0.806
Similaridade entre Usuários 2 e 2: 1
Similaridade entre Usuários 2 e 3: 0.673
Similaridade entre Usuários 3 e 0: 0.647
Similaridade entre Usuários 3 e 1: 0.64
Similaridade entre Usuários 3 e 2: 0.673
Similaridade entre Usuários 3 e 3: 1


#### 4. Matrix de Similaridades

Podemos imaginar uma outra matrix Usuários X Usuários, denominada **"Matrix de Similaridades"**, para guardar esses dados de similaridades.

In [7]:
matrix_similaridades = np.empty(NUM_USUARIOS, dtype=np.ndarray)

for i in range(NUM_USUARIOS):
    similar = []

    for j in range(NUM_USUARIOS):
        similar.append(calcular_similaridade(matrix_avaliacoes[i], matrix_avaliacoes[j]))

    matrix_similaridades[i] = np.array(similar)

matrix_similaridades = np.stack(matrix_similaridades)

In [8]:
print(matrix_similaridades)

[[1.         0.57522374 0.65346404 0.64748921]
 [0.57522374 1.         0.80636932 0.64044476]
 [0.65346404 0.80636932 1.         0.67322574]
 [0.64748921 0.64044476 0.67322574 1.        ]]


Outra maneira mais rápida de calcular essa matrix é a partir dessa linha de código:

In [9]:
matrix_similaridades = np.dot(matrix_avaliacoes, matrix_avaliacoes.T) / (np.linalg.norm(matrix_avaliacoes, axis=1)[:, None] * np.linalg.norm(matrix_avaliacoes.T, axis=0))

In [10]:
print(matrix_similaridades)

[[1.         0.57522374 0.65346404 0.64748921]
 [0.57522374 1.         0.80636932 0.64044476]
 [0.65346404 0.80636932 1.         0.67322574]
 [0.64748921 0.64044476 0.67322574 1.        ]]


Agora podemos utilizar essa matrix de similaridades para avaliar a utilidade de um item para um usuário.

#### 5. Avaliação de Utilidade

Vamos definir uma função denominada "Avaliar" que pega um usuário e um item, e retorna a Utilidade daquele item para aquele usuário.

In [11]:
def avaliar(usuario: int, item: int) -> float:
    """"
    Dado um usuário e um item, retorna a utilidade daquele item para o usuário.

    Args:
        usuario: int do id do usuário
        item: int do id do item
        k: int do número de usuarios similares usados na comparação

    Returns:
        utilidade: float correspondente a previsão da utilidade/avaliação do 
                   item para o usuário
    """

    # Número de usuários similares utilizados no cálculo da utilidade.
    k = 2

    # Primeiro pegamos o grau de similaridade de cada usuário com o usuário 
    # definido no argumento.
    similaridades = matrix_similaridades[usuario]

    # Agora precisamos obter o id dos k usuários mais similares ao usuário 
    # definido no argumento que avaliaram o item definido no argumento.

    # Rankear todos usuários com base no grau de similaridade, e obter o id dos 
    # usuários mais similares.
    usuarios_rankeados = list(np.argsort(similaridades)[::-1][1:])

    # Para cada usuário, se ele não avaliou o item definido no argumento,
    # remova-o da lista de usuários similares.
    for index, usuario_rankeado in enumerate(usuarios_rankeados):
        if matrix_avaliacoes[usuario_rankeado, item] == 0:
            usuarios_rankeados.pop(index)

    # Obter os k usuários mais similares que avaliaram o item definido 
    # no argumento.
    usuarios_mais_similares = usuarios_rankeados[:k]

    # Obter as avaliações desses usuários e o grau de similaridade de cada um.
    avaliacoes_similares_do_item = matrix_avaliacoes[usuarios_mais_similares, item]
    grau_similaridade = matrix_similaridades[usuario, usuarios_mais_similares]

    # Fazer uma média ponderada das avaliações, onde os pesos são os graus 
    # de similaridade.
    soma_ponderada = 0
    for avaliacao, grau in zip(avaliacoes_similares_do_item, grau_similaridade):
        soma_ponderada += grau * avaliacao 
    soma_pesos = sum(grau_similaridade)
    media_ponderada = soma_ponderada / soma_pesos

    # Definimos a utilidade como a média ponderada.
    utilidade = media_ponderada

    return utilidade

Agora podemos avaliar a utilidade de cada item para cada usuário:

In [12]:
for u in range(NUM_USUARIOS):
    for i in range(NUM_ITENS):
        print()
        print(f"Avaliando item {i} para usuário {u}: {avaliar(u, i):.3g}", end="")


Avaliando item 0 para usuário 0: 3
Avaliando item 1 para usuário 0: 3.94
Avaliando item 2 para usuário 0: 4
Avaliando item 3 para usuário 0: 4
Avaliando item 4 para usuário 0: 4.47
Avaliando item 0 para usuário 1: 2.89
Avaliando item 1 para usuário 1: 2.05
Avaliando item 2 para usuário 1: 3.89
Avaliando item 3 para usuário 1: 4.11
Avaliando item 4 para usuário 1: 3.17
Avaliando item 0 para usuário 2: 2.37
Avaliando item 1 para usuário 2: 4.09
Avaliando item 2 para usuário 2: 3.37
Avaliando item 3 para usuário 2: 4.09
Avaliando item 4 para usuário 2: 3.66
Avaliando item 0 para usuário 3: 3.47
Avaliando item 1 para usuário 3: 2.99
Avaliando item 2 para usuário 3: 2.51
Avaliando item 3 para usuário 3: 3.53
Avaliando item 4 para usuário 3: 3.02

Vamos criar um ranqueamento dessas avaliações, e recomendar os itens mais bem ranqueados para cada usuário, dado que o usuário já não avaliou aquele item.

In [13]:
recomendacoes = []

for u in range(NUM_USUARIOS):
    avaliacoes = []
    for i in range(NUM_ITENS):
        avaliacao = avaliar(u, i)
        if matrix_avaliacoes[u, i] != 0:
            continue
        else:
            avaliacoes.append((i, avaliacao))
    
    avaliacoes_ranqueadas = sorted(avaliacoes, key=(lambda x: x[1]))

    recomendacoes.append((u, avaliacoes_ranqueadas))

print("Recomendações\n")
for recomendacao in recomendacoes:
    print(f"Usuário {recomendacao[0]}: ", end='')

    if recomendacao[1]:
        for i in recomendacao[1]:
            print(f"Item {i[0]}", end='')
    
    print("")

Recomendações

Usuário 0: Item 2
Usuário 1: 
Usuário 2: Item 1
Usuário 3: Item 4
